# Task 1: Binary Representations

Trying to rotate bits to the left

In [4]:
def rotl(x, n=1):
    # Trying to rotate bits to the left
    return (x << n) | (x >> (32 - n))

# Test
print(bin(rotl(0b0001, 1)))


0b10
